In [ ]:
KUBECONFIG_PATH = "/home/junzhouh/distributed_debugger/SocialNetwork-serviceweaver/kubeconfig"
SERVICEWEAVER_POD_LABEL = "serviceweaver/app=server.out" # eg. "serviceweaver/app=server.out"

In [4]:

from kubernetes import client,config
config.load_kube_config(KUBECONFIG_PATH)
api_instance=client.CoreV1Api()

In [5]:
from uuid import uuid4
debug_container_name=f"ssh-debugger{uuid4()}"
debug_container_name

'ssh-debuggera0ab95cf-f495-49be-bf1e-896e546afe8c'

In [6]:
def setup_debug_container(pod_name: str, namespace: str = "default") -> None:

    debug_container = client.V1EphemeralContainer(
                    name=debug_container_name,
                    image="h21565897/distributeddebugger:144",
                    target_container_name="serviceweaver",
                    image_pull_policy="Always",
                    stdin=True,
                    tty=True,
                    security_context=client.V1SecurityContext(
                        privileged=True
                    )
                )
    patch_body = {
                "spec": {
                    "ephemeralContainers": [
                        debug_container
                    ]
                }
            }
            
            # Apply the patch
    api_instance.patch_namespaced_pod_ephemeralcontainers(
        name=pod_name,
        namespace="default",
        body=patch_body
    )

In [7]:
pods = api_instance.list_namespaced_pod(
            namespace="default", label_selector=SERVICEWEAVER_POD_LABEL)
len(pods.items)

14

In [8]:
for pod in pods.items:
    setup_debug_container(pod.metadata.name)
    print(f"Debug container injected for pod {pod.metadata.name}")

Debug container injected for pod scalingmachines-958e9cae-9106ee8d-5d594948f9-7lt7r
Debug container injected for pod server-backendservicer-958e9cae-586f2db3-5f8fb8df75-zfmf9
Debug container injected for pod server-ihometimelineservice-958e9cae-a9958526-84fb7f8dd5-ts2ss
Debug container injected for pod server-imediaservice-958e9cae-92e991b3-79dc7f8bd6-ln8s6
Debug container injected for pod server-isocialgraphservice-958e9cae-6f7f5686-5b7bb7d887-wzq6f
Debug container injected for pod server-itextservice-958e9cae-5453d508-6b774b6c87-qkjrk
Debug container injected for pod server-iuniqueidservice-958e9cae-e44cfbcd-5b9cc47799-jj67x
Debug container injected for pod server-iurlshortenservice-958e9cae-d2dab16f-58766597cd-tqjsz
Debug container injected for pod server-iusermentionservice-958e9cae-efebed60-5f4f5db5f6-qvx98
Debug container injected for pod server-iusertimelineservice-958e9cae-3a77e934-688dcd4fd9-7l4mb
Debug container injected for pod server-mediastorageservicer-958e9cae-0390fc14-5